# Read data and functions

In [1322]:
import geopandas as gpd
import folium
import rioxarray as riox
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.linear_model import LassoCV
from numpy import arange
from geopandas import GeoDataFrame, GeoSeries
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm

In [1281]:
pd.set_option('display.max_columns', None)

In [1282]:
neighborhoods_frankfurt = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\1 Land Prices\Land_Prices_Neighborhood_Frankfurt_am_Main.gpkg")
buildings_frankfurt= gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\4 Buildings\Buildings_Frankfurt_am_Main.gpkg")

In [1283]:
#satelite_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\Neighborhoods_Frankfurt_am_Main.gpkg")

In [1284]:
satelite_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\Neighborhoods_Frankfurt_am_Main_3.gpkg")

In [1285]:
dist_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\ffm_dist.gpkg")

In [1286]:
socio_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\zensus_ffm.gpkg")

In [1287]:
socio_data = socio_data.groupby('Neighborhood_FID').first().reset_index()
socio_data = socio_data.set_index('Neighborhood_FID')
dist_data = dist_data.set_index('Neighborhood_FID')
satelite_data = satelite_data.set_index('Neighborhood_FID')
neighborhoods_frankfurt =neighborhoods_frankfurt.set_index('Neighborhood_FID')

In [1288]:
sat_col = list(satelite_data)

In [1289]:
socio_col = list(socio_data)

# Construct Variables - stored in df with neighborhoods in the rows
interesting: 
neighborhoods_frankfurt['Area_Types'] - create dummies whether mixed, residential, commercial...
buildings_frankfurt['Building_Type'] - residential apartments share, office buildings, indicator whether in 75th percentile in terms of amount of ofice buildings, number of sports_centers, 
satelite_data['treecover']
satelite_data['builtup']
satelite_data['shrubland']
satelite_data['grassland']

### satelite data and neighborhoods are already on neighborhood level, buildings is not. 

In [1290]:
neigh_level_vars = gpd.GeoDataFrame()
neigh_level_vars['Land_Value']= neighborhoods_frankfurt['Land_Value']
neigh_level_vars['treecover']= satelite_data['treecover']
neigh_level_vars['builtup']= satelite_data['builtup']
neigh_level_vars['shrubland']= satelite_data['shrubland']
neigh_level_vars['grassland']= satelite_data['grassland']
neigh_level_vars['affected_07']= satelite_data['affected_07']
neigh_level_vars['affected_25']= satelite_data['affected_25']
neigh_level_vars[sat_col[7:]]=satelite_data[sat_col[7:]]
neigh_level_vars[socio_col[-9:]]=socio_data[socio_col[-9:]]
neigh_level_vars['dist']= dist_data['dist']


In [1291]:
neigh_level_vars= neigh_level_vars[:-1]

In [1292]:
neigh_level_vars.isnull().sum()

Land_Value                    0
treecover                     0
builtup                       0
shrubland                     0
grassland                     0
affected_07                  29
affected_25                  41
NUM_subway                    0
NUM_railway                   0
NUM_shops                     0
NUM_hospitals                 0
NUM_schools                   0
NUM_parks                     0
geometry                      0
area                          0
pop_nbh                       0
gerborn_tot_nbh               0
mig_share_nbh                 0
pop_density                   0
hh_child_couple_nbh           0
hh_child_single_nbh           0
hh_child_single_share_nbh     1
dist                          0
dtype: int64

In [1293]:
satelite_data[satelite_data['builtup'].isnull()]

,Neighborhood_Name,City_Code,City_Name,treecover,shrubland,grassland,builtup,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,affected_07,affected_25,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,


### normalize variables

In [1294]:
neigh_level_vars['area']= neigh_level_vars['area']*10000

In [1295]:
for element in ['NUM_shops', 'NUM_railway', 'NUM_subway', 'NUM_hospitals', 'NUM_schools', 'NUM_parks']: 
    
    neigh_level_vars[element]= neigh_level_vars[element]/neigh_level_vars['area'].values

### type of buildings - shares


### residential and non-residential

In [1296]:
buildings_frankfurt['resid']= np.where(
    buildings_frankfurt['Building_TypeGen'] == 'residential', 1, np.where(
    buildings_frankfurt['Building_TypeGen'] == 'non-residential' , 0, 0)) 

In [1297]:
buildings_frankfurt['nresid']= np.where(
    buildings_frankfurt['Building_TypeGen'] == 'non-residential', 1, np.where(
    buildings_frankfurt['Building_TypeGen'] == 'residential' , 0, 0)) 

In [1298]:
neighb_aggr2 = buildings_frankfurt.pivot_table(values = 'resid', index = 'Neighborhood_FID', columns = 'Build_ID')
neighb_aggr2['resid']=neighb_aggr2.sum(axis=1)
neigh_level_vars['resid'] = neighb_aggr2['resid']

In [1299]:
neighb_aggr3 = buildings_frankfurt.pivot_table(values = 'nresid', index = 'Neighborhood_FID', columns = 'Build_ID')
neighb_aggr3['nresid']=neighb_aggr3.sum(axis=1)
neigh_level_vars['nresid'] = neighb_aggr3['nresid']

In [1300]:
neigh_level_vars['build_count']= neigh_level_vars['resid'] + neigh_level_vars['nresid']
neigh_level_vars['resid_share']= neigh_level_vars['resid']/neigh_level_vars['build_count'].values

### average building height


In [1301]:
neighb_aggr = buildings_frankfurt.pivot_table(values = 'Building_Height', index = 'Neighborhood_FID', columns = 'Build_ID')
neighb_aggr['avg_height']=neighb_aggr.sum(axis=1)
neigh_level_vars['avg_height']= neighb_aggr['avg_height']

### number of houses

also interesting: the share of houses vs. apartments - number of people per square meter

In [1302]:
neighb_aggr = gpd.GeoDataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='house'].groupby('Neighborhood_FID').count()
neigh_level_vars['house_count']= neighb_aggr['1']

In [1303]:
neighb_aggr = gpd.GeoDataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='apartments'].groupby('Neighborhood_FID').count()
neigh_level_vars['apartment_count']= neighb_aggr['1']

In [1304]:
neighb_aggr = gpd.GeoDataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='office'].groupby('Neighborhood_FID').count()
neighb_aggr_final['office_count']= neighb_aggr['1']

## type of Area

In [1305]:
test = gpd.GeoDataFrame
test = neighborhoods_frankfurt['Area_Types'].str.split(pat='_',expand=True)

In [1306]:
neigh_level_vars[['areatype_1', 'areatype_2', 'areatype_3']]= test[[0, 1, 2]]

In [1307]:
neigh_level_vars['areatype_1'].value_counts()

areatype_1
AgriculturalArea          16
CommercialBuildingArea    15
MixedUseArea              13
OtherArea                  1
Name: count, dtype: int64

In [1308]:
neigh_level_vars['areatype_agri'] = np.where(neigh_level_vars['areatype_1']=='AgriculturalArea', 1, 0)
neigh_level_vars['areatype_commer'] = np.where(neigh_level_vars['areatype_1']=='CommercialBuildingArea', 1, 0)
neigh_level_vars['areatype_mixed'] = np.where(neigh_level_vars['areatype_1']=='MixedUseArea', 1, 0)

In [1309]:
neigh_level_vars = neigh_level_vars.drop([ 'areatype_1','areatype_2', 'areatype_3', 'resid', 'nresid'], axis=1)

In [1310]:
neigh_level_vars['house_count'].fillna(neigh_level_vars['house_count'].mean(), inplace=True)
neigh_level_vars['apartment_count'].fillna(neigh_level_vars['apartment_count'].mean(), inplace=True)

neigh_level_vars['hh_child_single_share_nbh'].fillna(neigh_level_vars.iloc[[13, 17],18].mean(), inplace=True)
neigh_level_vars = neigh_level_vars.drop('geometry', axis=1)

In [1311]:
neigh_level_vars['affected_07'].fillna(0, inplace=True)
neigh_level_vars['affected_25'].fillna(0, inplace=True)

In [1312]:
colnames=list(neigh_level_vars.columns)

# Predictions

# Lasso

In [1314]:
neigh_level_vars.isnull().sum()

Land_Value                   0
treecover                    0
builtup                      0
shrubland                    0
grassland                    0
affected_07                  0
affected_25                  0
NUM_subway                   0
NUM_railway                  0
NUM_shops                    0
NUM_hospitals                0
NUM_schools                  0
NUM_parks                    0
area                         0
pop_nbh                      0
gerborn_tot_nbh              0
mig_share_nbh                0
pop_density                  0
hh_child_couple_nbh          0
hh_child_single_nbh          0
hh_child_single_share_nbh    0
dist                         0
build_count                  0
resid_share                  0
avg_height                   0
house_count                  0
apartment_count              0
areatype_agri                0
areatype_commer              0
areatype_mixed               0
dtype: int64

In [1315]:
model_LASSO = Lasso(alpha=0.5)
#model = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# force scores to be positive
#scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

#withprice2= withprice[['office_perc75', 'resid_share', 'Land_Value']]
data = neigh_level_vars.values


In [1316]:
X_LASSO, y_LASSO = data[:,1: ], data[:, 0]
# evaluate model
scores = cross_val_score(model_LASSO, X_LASSO, y_LASSO, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
model_LASSO.fit(X_LASSO,y_LASSO)


C:\Users\nb\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.647e+07, tolerance: 2.646e+04
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.5)

In [1375]:
results_prices= gpd.GeoDataFrame()
results_prices['Lasso_pred']= model_LASSO.predict(X_LASSO)

# OLS

In [1422]:
coefficients = list(neigh_level_vars)
coefficients= coefficients[1:]
X_OLS = neigh_level_vars[coefficients]
y_OLS = neigh_level_vars['Land_Value']

In [1423]:
print(coefficients)

['treecover', 'builtup', 'shrubland', 'grassland', 'affected_07', 'affected_25', 'NUM_subway', 'NUM_railway', 'NUM_shops', 'NUM_hospitals', 'NUM_schools', 'NUM_parks', 'area', 'pop_nbh', 'gerborn_tot_nbh', 'mig_share_nbh', 'pop_density', 'hh_child_couple_nbh', 'hh_child_single_nbh', 'hh_child_single_share_nbh', 'dist', 'build_count', 'resid_share', 'avg_height', 'house_count', 'apartment_count', 'areatype_agri', 'areatype_commer', 'areatype_mixed']


In [1424]:
regr = linear_model.LinearRegression()
regr.fit(X_OLS, y_OLS)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 343.3383419533943
Coefficients: 
 [-6.49450374e+03 -2.58531012e+03 -2.47867902e+07 -3.83510520e+03
 -1.46990084e+03  5.75321311e+02  3.54863542e+06  2.15040486e+07
 -1.42658132e+06 -9.79087381e+06 -1.23912403e+06  1.77768133e+06
  4.19351471e-04 -9.64690429e-01  1.72459272e+00  1.79555084e+04
  9.39465486e-02 -4.31658576e-01 -3.77581493e+00  4.79588450e+00
 -3.95538592e-01  9.36502929e-01 -1.10698732e+02  4.79665104e-02
 -3.51021720e+00 -1.67656163e+00 -1.51143985e+02  6.23336652e+02
  6.25905341e+02]


In [1338]:
coefs=regr.coef_
#(pd.DataFrame({'coef':regr.coef_, 'category':X.columns}).set_index('category'))

#print(coefs)

In [1347]:
np.savetxt("coefs_test.csv", coefs, delimiter=",")

In [1345]:
coefs.to_csv(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\coefs_streamlit_test.csv", sep=',')

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [1376]:
results_prices['OLS']= predictions_OLS

In [1377]:
resetindex= gpd.GeoDataFrame()
resetindex['Land_Value']=neigh_level_vars['Land_Value']
resetindex = resetindex.reset_index()

In [1378]:
results_prices['True']= resetindex['Land_Value']

In [1379]:
results_prices['err_lasso']= abs(results_prices['Lasso_pred']-results_prices['True'])
results_prices['err_ols']= abs(results_prices['OLS']-results_prices['True'])

In [1380]:
print('Lasso mean prediction error: ',results_prices['err_lasso'].sum()/len(results_prices))
print('OLS mean prediction error: ',results_prices['err_ols'].sum()/len(results_prices))

Lasso mean prediction error:  679.964548195824
OLS mean prediction error:  1033.9089815134153


In [1381]:
results_prices['True'].mean()

2427.237777777778

In [1393]:
satelite_data.reset_index()

,Neighborhood_FID,Neighborhood_Name,City_Code,City_Name,treecover,shrubland,grassland,builtup,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,affected_07,affected_25,geometry
0,1,Altstadt,06412000,Frankfurt am Main,0.133689,0.000000,0.009084,0.729463,0.0,0.0,4.0,0.0,0.0,5.0,NaN,NaN,"MULTIPOLYGON (((4226384.447 3000227.322, 42263..."
1,2,Innenstadt,06412000,Frankfurt am Main,0.249146,0.000000,0.008905,0.699297,7.0,3.0,9.0,1.0,3.0,18.0,NaN,NaN,"MULTIPOLYGON (((4227082.463 3001711.312, 42272..."
2,3,Bahnhofsviertel,06412000,Frankfurt am Main,0.084468,0.000000,0.006073,0.840633,0.0,0.0,13.0,0.0,2.0,6.0,NaN,NaN,"MULTIPOLYGON (((4225284.051 3000486.630, 42252..."
3,4,Westend-Süd,06412000,Frankfurt am Main,0.308698,0.000000,0.025584,0.658058,2.0,2.0,3.0,0.0,7.0,30.0,NaN,NaN,"MULTIPOLYGON (((4225219.883 3002076.156, 42252..."
4,5,Westend-Nord,06412000,Frankfurt am Main,0.478802,0.000000,0.095331,0.420782,2.0,0.0,6.0,0.0,7.0,15.0,NaN,NaN,"MULTIPOLYGON (((4225993.384 3003603.082, 42260..."
5,6,Nordend-West,06412000,Frankfurt am Main,0.422453,0.000000,0.036121,0.539943,6.0,0.0,11.0,2.0,11.0,21.0,NaN,NaN,"MULTIPOLYGON (((4227858.464 3003565.575, 42278..."
6,7,Nordend-Ost,06412000,Frankfurt am Main,0.447289,0.000000,0.026845,0.524233,2.0,0.0,5.0,0.0,7.0,22.0,NaN,NaN,"MULTIPOLYGON (((4227867.212 3001590.060, 42277..."
7,8,Ostend,06412000,Frankfurt am Main,0.196036,0.000000,0.049009,0.656307,5.0,2.0,13.0,2.0,8.0,23.0,1.0,NaN,"MULTIPOLYGON (((4231428.842 3001758.583, 42314..."
8,9,Bornheim,06412000,Frankfurt am Main,0.447060,0.000000,0.065875,0.476517,3.0,0.0,12.0,2.0,7.0,9.0,NaN,NaN,"MULTIPOLYGON (((4228434.896 3001994.720, 42284..."
9,10,Gutleutviertel,06412000,Frankfurt am Main,0.172629,0.000000,0.036180,0.649334,0.0,0.0,1.0,0.0,2.0,6.0,1.0,NaN,"MULTIPOLYGON (((4222367.116 2998542.440, 42223..."


In [1396]:
test2 = pd.DataFrame()
test2 = satelite_data

test2 = test2.reset_index()

In [1399]:
results_prices['Neighborhood']= test2['Neighborhood_Name']

In [1401]:
results_prices.set_index('Neighborhood')

,Lasso_pred,OLS,True,err_lasso,err_ols
Neighborhood,,,,,
Altstadt,6849.175815,6993.088407,6998.5,149.324185,5.411593
Innenstadt,7113.991447,11088.353478,11756.9,4642.908553,668.546522
Bahnhofsviertel,8048.014678,4781.992362,5312.2,2735.814678,530.207638
Westend-Süd,7515.006753,9598.300016,9779.6,2264.593247,181.299984
Westend-Nord,4054.228221,4214.722733,4236.2,181.971779,21.477267
Nordend-West,6652.057238,6770.192935,6407.0,245.057238,363.192935
Nordend-Ost,5221.080409,5196.817275,5529.3,308.219591,332.482725
Ostend,3720.159938,3588.924905,4334.1,613.940062,745.175095
Bornheim,3751.359628,3432.671399,3302.5,448.859628,130.171399


In [1404]:
results_prices.to_csv(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\predictions_errors_lasso_ols.csv", sep=',')

In [1430]:
def model_fit_OLS(X_OLS, y_OLS):
    regr = linear_model.LinearRegression()
    regr.fit(X_OLS, y_OLS)
    return regr
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)

In [1431]:
def model_fit_LASSO(X_LASSO, y_LASSO):
    model_LASSO = Lasso(alpha=0.5)
    model_LASSO.fit(X_LASSO,y_LASSO)
    return model_LASSO

In [1432]:
model_LASSO

Lasso(alpha=0.5)

In [1434]:
def model_predict_OLS(regr, X_test_OLS):
	y_pred = regr.predict(X_test)
	return y_pred

In [1438]:
def model_predict_LASSO(regr, X_test_LASSO):
	y_pred_LASSO = model_LASSO.predict(X_test_LASSO)
	return y_pred_LASSO

# Export data

In [1157]:
neigh_level_vars_streamlite=gpd.GeoDataFrame()
neigh_level_vars_streamlite = neigh_level_vars
#neigh_level_vars_streamlite.drop(neigh_level_vars_streamlite['Land_Value'], axis=1)

In [1171]:
neigh_level_vars_streamlite.drop('Land_Value', axis=1)

,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.133689,0.729463,0.000000,0.009084,0.0,0.0,0.000000,0.000000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.000000,72.0,0.791667,954.5,4.000000,48.000000,0,0,1,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4..."
2,0.249146,0.699297,0.000000,0.008905,0.0,0.0,0.000467,0.000200,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.000000,194.0,0.350515,2907.1,3.000000,64.000000,0,0,1,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4..."
3,0.084468,0.840633,0.000000,0.006073,0.0,0.0,0.000000,0.000000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.000000,130.0,0.400000,1693.5,1.000000,50.000000,0,0,1,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4..."
4,0.308698,0.658058,0.000000,0.025584,0.0,0.0,0.000080,0.000080,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.000000,480.0,0.575000,4654.8,1.000000,263.000000,0,1,0,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4..."
5,0.478802,0.420782,0.000000,0.095331,0.0,0.0,0.000122,0.000000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.484170,647.0,0.761978,5052.0,17.000000,421.000000,0,0,1,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4..."
6,0.422453,0.539943,0.000000,0.036121,0.0,0.0,0.000193,0.000000,0.000353,0.000064,0.000353,0.000675,31132.536753,24706,18798,0.239132,7935.749083,2075,778,0.272695,0.000000,1319.0,0.822593,7173.5,11.000000,1060.000000,0,0,1,"MULTIPOLYGON (((4227858.46392 3003565.57485, 4..."
7,0.447289,0.524233,0.000000,0.026845,0.0,0.0,0.000130,0.000000,0.000325,0.000000,0.000455,0.001430,15381.550793,16946,12891,0.239290,11017.094589,1351,610,0.311066,0.000000,1403.0,0.908767,3291.5,143.000000,1117.000000,0,1,0,"MULTIPOLYGON (((4227867.21204 3001590.05997, 4..."
8,0.196036,0.656307,0.000000,0.049009,1.0,0.0,0.000090,0.000036,0.000233,0.000036,0.000143,0.000412,55848.432463,22410,14706,0.343775,4012.646195,1725,702,0.289246,0.000000,2163.0,0.714748,10443.5,38.000000,1476.000000,0,1,0,"MULTIPOLYGON (((4231428.84153 3001758.58257, 4..."
9,0.447060,0.476517,0.000000,0.065875,0.0,0.0,0.000107,0.000000,0.000429,0.000071,0.000250,0.000322,27978.401596,25316,18316,0.276505,9048.408256,1974,945,0.323741,864.461323,2969.0,0.807343,16737.5,362.000000,2028.000000,0,0,1,"MULTIPOLYGON (((4228434.89586 3001994.71993, 4..."


In [1420]:
gdf= gpd.GeoDataFrame(neigh_level_vars_streamlite)
gdf.drop('geometry', axis=1)
gdf.head()

,Land_Value,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6998.5,0.133689,0.729463,0.0,0.009084,0.0,0.0,0.000000,0.00000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.00000,72.0,0.791667,954.5,4.0,48.0,0,0,1,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4..."
2,11756.9,0.249146,0.699297,0.0,0.008905,0.0,0.0,0.000467,0.00020,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.00000,194.0,0.350515,2907.1,3.0,64.0,0,0,1,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4..."
3,5312.2,0.084468,0.840633,0.0,0.006073,0.0,0.0,0.000000,0.00000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.00000,130.0,0.400000,1693.5,1.0,50.0,0,0,1,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4..."
4,9779.6,0.308698,0.658058,0.0,0.025584,0.0,0.0,0.000080,0.00008,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.00000,480.0,0.575000,4654.8,1.0,263.0,0,1,0,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4..."
5,4236.2,0.478802,0.420782,0.0,0.095331,0.0,0.0,0.000122,0.00000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.48417,647.0,0.761978,5052.0,17.0,421.0,0,0,1,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4..."


In [1419]:
gdf['geometry']=satelite_data['geometry']
gdf['Neighborhood']=satelite_data['Neighborhood_Name']
gdf['flugschneise2']= gdf['affected_07']+gdf['affected_25']
gdf['flugschneise2'][(gdf['affected_07']==1) & (gdf['affected_25']==1)] = 1
gdf['flugschneise2'][(gdf['affected_07']==1) | (gdf['affected_25']==1)] = 1
gdf['flugschneise2'][(gdf['affected_07']!=1) & (gdf['affected_25']!=1)] = 0

gdf.drop(['affected_07', 'affected_25'], axis=1)
gdf['flugschneise']= gdf['flugschneise2']
gdf.drop('flugschneise2', axis=1)

C:\Users\nb\AppData\Local\Temp\ipykernel_7020\3574790402.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['flugschneise2'][(gdf['affected_07']==1) & (gdf['affected_25']==1)] = 1
C:\Users\nb\AppData\Local\Temp\ipykernel_7020\3574790402.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['flugschneise2'][(gdf['affected_07']==1) | (gdf['affected_25']==1)] = 1
C:\Users\nb\AppData\Local\Temp\ipykernel_7020\3574790402.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,Land_Value,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed,geometry,Neighborhood,flugschneise
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6998.5,0.133689,0.729463,0.000000,0.009084,0.0,0.0,0.000000,0.000000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.000000,72.0,0.791667,954.5,4.000000,48.000000,0,0,1,"MULTIPOLYGON (((4227082.463 3001711.312, 42272...",Innenstadt,0.0
2,11756.9,0.249146,0.699297,0.000000,0.008905,0.0,0.0,0.000467,0.000200,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.000000,194.0,0.350515,2907.1,3.000000,64.000000,0,0,1,"MULTIPOLYGON (((4225284.051 3000486.630, 42252...",Bahnhofsviertel,0.0
3,5312.2,0.084468,0.840633,0.000000,0.006073,0.0,0.0,0.000000,0.000000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.000000,130.0,0.400000,1693.5,1.000000,50.000000,0,0,1,"MULTIPOLYGON (((4225219.883 3002076.156, 42252...",Westend-Süd,0.0
4,9779.6,0.308698,0.658058,0.000000,0.025584,0.0,0.0,0.000080,0.000080,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.000000,480.0,0.575000,4654.8,1.000000,263.000000,0,1,0,"MULTIPOLYGON (((4225993.384 3003603.082, 42260...",Westend-Nord,0.0
5,4236.2,0.478802,0.420782,0.000000,0.095331,0.0,0.0,0.000122,0.000000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.484170,647.0,0.761978,5052.0,17.000000,421.000000,0,0,1,"MULTIPOLYGON (((4227858.464 3003565.575, 42278...",Nordend-West,0.0
6,6407.0,0.422453,0.539943,0.000000,0.036121,0.0,0.0,0.000193,0.000000,0.000353,0.000064,0.000353,0.000675,31132.536753,24706,18798,0.239132,7935.749083,2075,778,0.272695,0.000000,1319.0,0.822593,7173.5,11.000000,1060.000000,0,0,1,"MULTIPOLYGON (((4227867.212 3001590.060, 42277...",Nordend-Ost,0.0
7,5529.3,0.447289,0.524233,0.000000,0.026845,0.0,0.0,0.000130,0.000000,0.000325,0.000000,0.000455,0.001430,15381.550793,16946,12891,0.239290,11017.094589,1351,610,0.311066,0.000000,1403.0,0.908767,3291.5,143.000000,1117.000000,0,1,0,"MULTIPOLYGON (((4231428.842 3001758.583, 42314...",Ostend,0.0
8,4334.1,0.196036,0.656307,0.000000,0.049009,1.0,0.0,0.000090,0.000036,0.000233,0.000036,0.000143,0.000412,55848.432463,22410,14706,0.343775,4012.646195,1725,702,0.289246,0.000000,2163.0,0.714748,10443.5,38.000000,1476.000000,0,1,0,"MULTIPOLYGON (((4228434.896 3001994.720, 42284...",Bornheim,1.0
9,3302.5,0.447060,0.476517,0.000000,0.065875,0.0,0.0,0.000107,0.000000,0.000429,0.000071,0.000250,0.000322,27978.401596,25316,18316,0.276505,9048.408256,1974,945,0.323741,864.461323,2969.0,0.807343,16737.5,362.000000,2028.000000,0,0,1,"MULTIPOLYGON (((4222367.116 2998542.440, 42223...",Gutleutviertel,0.0


In [1414]:
gdf.to_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\streamlit3.gpkg", driver="GPKG" )

# DONE